<a href="https://colab.research.google.com/github/AgilAptanaDwiPutra/ProjectMachineLearning/blob/main/Tugas_Machine_Learning_1_Model_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Memuat Dataset dan Pra-Pemrosesan Teks

In [2]:
# Import library yang diperlukan
import pandas as pd

# Memuat data dari file Excel
file_path = 'hasil_labelling.xlsx'  # Ganti dengan path file Anda
df = pd.read_excel(file_path)

# Menampilkan 5 baris pertama data untuk melihat struktur data
df.head()

,subtitle,label
0,Presiden AS Donald Trump mengisyaratkan akan a...,politik
1,Netanyahu menyebut negara Palestina akan menja...,others
2,Netanyahu mengambil penerbangan lebih panjang ...,politik
3,Spanyol akan bergabung dengan Italia dalam men...,politik
4,Media terkemuka Israel menyoroti pidato Presid...,politik


Preprocessing Teks

In [4]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 19.0 MB/s eta 0:00:00


In [5]:
# Import stopwords dari NLTK dan Sastrawi untuk stemming
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

# Inisialisasi stopwords untuk Bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Inisialisasi stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi pra-pemrosesan teks lengkap
def preprocess_text(text):
    # 1. Mengubah teks menjadi huruf kecil
    text = text.lower()

    # 2. Menghapus angka dan karakter non-alfabet (tanda baca, simbol)
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenisasi: Memecah teks menjadi kata-kata
    tokens = text.split()

    # 4. Menghapus stop words (kata yang tidak informatif)
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Stemming: Mengubah kata menjadi bentuk dasarnya menggunakan Sastrawi
    tokens = [stemmer.stem(word) for word in tokens]

    # 6. Gabungkan token menjadi satu string lagi
    clean_text = " ".join(tokens)

    return clean_text

# Terapkan pra-pemrosesan ke kolom subtitle
df['clean_text'] = df['subtitle'].apply(preprocess_text)

# Menampilkan 5 baris pertama setelah pra-pemrosesan
df[['subtitle', 'clean_text']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,subtitle,clean_text
0,Presiden AS Donald Trump mengisyaratkan akan a...,presiden as donald trump isyarat terobos krisi...
1,Netanyahu menyebut negara Palestina akan menja...,netanyahu sebut negara palestina bunuh nasiona...
2,Netanyahu mengambil penerbangan lebih panjang ...,netanyahu ambil terbang new york hadir sidang ...
3,Spanyol akan bergabung dengan Italia dalam men...,spanyol gabung italia kirim kapal perang lindu...
4,Media terkemuka Israel menyoroti pidato Presid...,media muka israel sorot pidato presiden ri pra...


Pembagian Data untuk Pelatihan dan Pengujian

In [10]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 41.1 MB/s eta 0:00:00


In [6]:
# Memisahkan data menjadi train dan test (80% train, 20% test)
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=42)

# Menampilkan beberapa contoh setelah pemisahan
train_texts.head(), test_texts.head()

(65      as baterai sistem tahan rudal terminal high al...
 5296           perintah jepang seru warga tinggal lebanon
 8726    pesawat militer rusia jatuh wilayah belgorod r...
 3254    awat rumah sakit dr agung bima ntb ade saputra...
 5858    xi jinping selamat donald trump menang pilpres...
 Name: clean_text, dtype: object,
 6661    heru budi orang tugas pecat jarah aset bangun ...
 5474    presiden palestina mahmoud abbas respons aksi ...
 8402    remaja lakilaki rusia juluk pahlawan selamat o...
 6582    suami bunga citra lestari bcl tiko pradipta ar...
 5402    pasu amerika serikat as tewas pimpin senior ke...
 Name: clean_text, dtype: object)

Menggunakan Fitur BoW, TF-IDF, dan Word2Vec

In [7]:
# Import CountVectorizer untuk BoW
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words Vectorization
bow_vectorizer = CountVectorizer(max_features=1000)
X_train_bow = bow_vectorizer.fit_transform(train_texts)
X_test_bow = bow_vectorizer.transform(test_texts)

# Menampilkan hasil fitur BoW
X_train_bow.toarray()[:5]  # Menampilkan 5 baris pertama

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
# Import TfidfVectorizer untuk TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

# Menampilkan hasil fitur TF-IDF
X_train_tfidf.toarray()[:5]  # Menampilkan 5 baris pertama

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
import gensim
import numpy as np

# Preprocessing untuk Word2Vec: Tokenisasi
train_tokens = [text.split() for text in train_texts]
test_tokens = [text.split() for text in test_texts]

# Melatih model Word2Vec
word2vec_model = gensim.models.Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Menghitung rata-rata vektor untuk setiap kalimat
def get_average_word2vec(tokens_list, model, vector_size):
    vector = np.zeros(vector_size)
    count = 0
    for token in tokens_list:
        try:
            vector += model.wv[token]
            count += 1
        except KeyError:  # Jika kata tidak ada dalam vocabulary
            continue
    if count > 0:
        vector /= count
    return vector

X_train_word2vec = np.array([get_average_word2vec(tokens, word2vec_model, 100) for tokens in train_tokens])
X_test_word2vec = np.array([get_average_word2vec(tokens, word2vec_model, 100) for tokens in test_tokens])

# Menampilkan hasil fitur Word2Vec
X_train_word2vec[:5]  # Menampilkan 5 baris pertama

array([[-0.14652228,  0.24044292,  0.21887614, -0.01506963,  0.10769527,
        -0.64650747,  0.19869406,  0.6331296 , -0.26932919, -0.11313689,
        -0.2396016 , -0.45094936, -0.0846263 ,  0.14045555,  0.24415177,
        -0.25806316,  0.05729422, -0.44264243, -0.03165194, -0.7245062 ,
         0.11095935,  0.22816798,  0.19503239, -0.29090749, -0.13011183,
        -0.03051619, -0.26751324, -0.17779353, -0.32420511, -0.07741274,
         0.34100875,  0.05571994,  0.17525217, -0.20785293, -0.1020992 ,
         0.34800241, -0.03642076, -0.1725725 , -0.19440046, -0.6585645 ,
         0.05987601, -0.25433601, -0.10018533,  0.0819657 ,  0.26846344,
        -0.18626639, -0.30716263, -0.01974813,  0.15323536,  0.30211943,
         0.00919358, -0.33135805, -0.1188811 , -0.05307339, -0.25727978,
         0.19970606,  0.06319435, -0.00383979, -0.37608893,  0.10318791,
         0.12263024,  0.18961448, -0.14585683,  0.01291256, -0.40662691,
         0.32956714,  0.19522643,  0.16820748, -0.5

Menggabungkan Fitur IndoBERT dengan BoW, TF-IDF, dan Word2Vec

In [13]:
# Fungsi untuk mendapatkan representasi IndoBERT
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=len(df['label'].unique()))

def get_indobert_features(dataset, model, tokenizer):
    features = []
    model.eval()
    with torch.no_grad():
        for text in dataset:
            inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
            outputs = model(**inputs)
            features.append(outputs.logits.cpu().numpy())
    return np.concatenate(features)

# Mendapatkan fitur IndoBERT
X_train_indobert = get_indobert_features(train_texts.tolist(), model, tokenizer)
X_test_indobert = get_indobert_features(test_texts.tolist(), model, tokenizer)

# Gabungkan IndoBERT dengan BoW
X_train_combined_bow = np.hstack((X_train_indobert, X_train_bow.toarray()))
X_test_combined_bow = np.hstack((X_test_indobert, X_test_bow.toarray()))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [14]:
# Gabungkan IndoBERT dengan TF-IDF
X_train_combined_tfidf = np.hstack((X_train_indobert, X_train_tfidf.toarray()))
X_test_combined_tfidf = np.hstack((X_test_indobert, X_test_tfidf.toarray()))

In [15]:
# Gabungkan IndoBERT dengan Word2Vec
X_train_combined_word2vec = np.hstack((X_train_indobert, X_train_word2vec))
X_test_combined_word2vec = np.hstack((X_test_indobert, X_test_word2vec))

Latih Model untuk IndoBERT + BoW

In [19]:
from sklearn.linear_model import LogisticRegression

# Melatih model Logistic Regression dengan fitur gabungan IndoBERT + BoW
lr_bow = LogisticRegression(max_iter=2000)
lr_bow.fit(X_train_combined_bow, train_labels)

print("Model IndoBERT + BoW berhasil dilatih.")

Model IndoBERT + BoW berhasil dilatih.


Latih Model untuk IndoBERT + TF-IDF

In [20]:
# Melatih model Logistic Regression dengan fitur gabungan IndoBERT + TF-IDF
lr_tfidf = LogisticRegression(max_iter=2000)
lr_tfidf.fit(X_train_combined_tfidf, train_labels)

print("Model IndoBERT + TF-IDF berhasil dilatih.")

Model IndoBERT + TF-IDF berhasil dilatih.


Latih Model untuk IndoBERT + Word2Vec

In [21]:
# Melatih model Logistic Regression dengan fitur gabungan IndoBERT + Word2Vec
lr_word2vec = LogisticRegression(max_iter=2000)
lr_word2vec.fit(X_train_combined_word2vec, train_labels)

print("Model IndoBERT + Word2Vec berhasil dilatih.")

Model IndoBERT + Word2Vec berhasil dilatih.


Evaluasi IndoBERT + BoW

In [23]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluasi IndoBERT + BoW
y_pred_bow = lr_bow.predict(X_test_combined_bow)

print("=== Evaluasi IndoBERT + BoW ===")
print("Accuracy:", accuracy_score(test_labels, y_pred_bow))
print(classification_report(test_labels, y_pred_bow))

=== Evaluasi IndoBERT + BoW ===
Accuracy: 0.8217299578059072
              precision    recall  f1-score   support

     hiburan       0.00      0.00      0.00        13
      others       0.78      0.86      0.82       864
     politik       0.96      0.95      0.95       583
   teknologi       0.71      0.59      0.65       436

    accuracy                           0.82      1896
   macro avg       0.61      0.60      0.61      1896
weighted avg       0.81      0.82      0.82      1896



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluasi IndoBERT + TF-IDF

In [24]:
# Evaluasi IndoBERT + TF-IDF
y_pred_tfidf = lr_tfidf.predict(X_test_combined_tfidf)

print("=== Evaluasi IndoBERT + TF-IDF ===")
print("Accuracy:", accuracy_score(test_labels, y_pred_tfidf))
print(classification_report(test_labels, y_pred_tfidf))

=== Evaluasi IndoBERT + TF-IDF ===
Accuracy: 0.8106540084388185
              precision    recall  f1-score   support

     hiburan       0.00      0.00      0.00        13
      others       0.76      0.89      0.82       864
     politik       0.93      0.93      0.93       583
   teknologi       0.76      0.53      0.62       436

    accuracy                           0.81      1896
   macro avg       0.61      0.59      0.59      1896
weighted avg       0.81      0.81      0.80      1896



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluasi IndoBERT + Word2Vec

In [25]:
# Evaluasi IndoBERT + Word2Vec
y_pred_word2vec = lr_word2vec.predict(X_test_combined_word2vec)

print("=== Evaluasi IndoBERT + Word2Vec ===")
print("Accuracy:", accuracy_score(test_labels, y_pred_word2vec))
print(classification_report(test_labels, y_pred_word2vec))

=== Evaluasi IndoBERT + Word2Vec ===
Accuracy: 0.6376582278481012
              precision    recall  f1-score   support

     hiburan       0.00      0.00      0.00        13
      others       0.59      0.84      0.70       864
     politik       0.73      0.78      0.75       583
   teknologi       0.61      0.06      0.11       436

    accuracy                           0.64      1896
   macro avg       0.48      0.42      0.39      1896
weighted avg       0.64      0.64      0.57      1896



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
